## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page [Medical LLM - 14B ](https://aws.amazon.com/marketplace/pp/prodview-u5vx4onx5kucy)
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

**Model**: `JSL-Medical-LLM-14B`  
**Model Description**: Medical LLM built for advanced clinical reasoning, complex medical case analysis, and deep comprehension of biomedical literature. Optimized for summarization, clinical question answering, and retrieval-augmented generation (RAG), it delivers high-accuracy responses in critical healthcare applications where depth and precision are essential.

In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [ ]:
import os
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

In [3]:
model_name = "JSL-Medical-LLM-14B"

real_time_inference_instance_type = "ml.g5.12xlarge"
batch_transform_inference_instance_type = "ml.g5.12xlarge"

## 2. Create a deployable model from the model package.

In [4]:
model = ModelPackage(
    role=role, 
    model_package_arn=model_package_arn, 
    sagemaker_session=sagemaker_session, 
)

## Model Configuration Documentation  

### Default Configuration  
The container comes with the following default configurations:  

| Parameter                  | Default Value | Description                                                                   |  
|----------------------------|---------------|-------------------------------------------------------------------------------|  
| **`dtype`**                | `float16`     | Data type for model weights and activations                                   |  
| **`max_model_len`**        | `16,384`      | Maximum length for input and output combined (`input + output ≤ max_model_len`) |  
| **`tensor_parallel_size`** | `2`          | Number of GPUs used for distributed execution with tensor parallelism.                            |  
| **`host`**                 | `0.0.0.0`     | Host name                                                                     |  
| **`port`**                 | `8080`        | Port number                                                                   |  

### Hardcoded Settings  
The following settings are hardcoded in the container and cannot be changed:  

| Parameter       | Value           | Description                           |  
|-----------------|-----------------|---------------------------------------|  
| **`model`**     | `/opt/ml/model` | Model path where SageMaker mounts the model |  

### Configurable Environment Variables  
You can customize the vLLM server by setting environment variables when creating the model.  

**Any parameter from the [vLLM documentation](https://docs.vllm.ai/en/latest/serving/openai_compatible_server.html#vllm-serve) can be set using the corresponding environment variable with the `SM_VLLM_` prefix.**  

The container uses a script similar to the [SageMaker entrypoint example](https://docs.vllm.ai/en/latest/examples/sagemaker_entrypoint.html) from the vLLM documentation to convert environment variables to command-line arguments.  

---  

## Input Format  

### 1. Chat Completion  

#### Example Payload  
```json  
{  
    "model": "/opt/ml/model",  
    "messages": [  
        {"role": "system", "content": "You are a helpful medical assistant."},  
        {"role": "user", "content": "What should I do if I have a fever and body aches?"}  
    ],  
    "max_tokens": 1024,  
    "temperature": 0.7  
}  
```  

For additional parameters:  
- [ChatCompletionRequest](https://github.com/vllm-project/vllm/blob/v0.7.3/vllm/entrypoints/openai/protocol.py#L212)  
- [OpenAI's Chat API](https://platform.openai.com/docs/api-reference/chat/create)  

---  

### 2. Text Completion  

#### Single Prompt Example  
```json  
{  
    "model": "/opt/ml/model",  
    "prompt": "How can I maintain good kidney health?",  
    "max_tokens": 512,  
    "temperature": 0.6  
}  
```  

#### Multiple Prompts Example  
```json  
{  
    "model": "/opt/ml/model",  
    "prompt": [  
        "How can I maintain good kidney health?",  
        "What are the best practices for kidney care?"  
    ],  
    "max_tokens": 512,  
    "temperature": 0.6  
}  
```  

Reference:  
- [CompletionRequest](https://github.com/vllm-project/vllm/blob/v0.7.3/vllm/entrypoints/openai/protocol.py#L642)  
- [OpenAI's Completions API](https://platform.openai.com/docs/api-reference/completions/create)  

---  

### Important Notes:
- **Streaming Responses:** Add `"stream": true` to your request payload to enable streaming
- **Model Path Requirement:** Always set `"model": "/opt/ml/model"` (SageMaker's fixed model location)

## 3. Create an SageMaker Endpoint

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [ ]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type, 
    endpoint_name=model_name,
    model_data_download_timeout=3600
)

### 3.1 Real-time inference via Amazon SageMaker Endpoint

#### Initial setup

In [8]:
prompt1 = "How do emerging mRNA technologies compare to traditional vaccine approaches for disease prevention?"

prompt2 = """A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus.

Which of the following is the best treatment for this patient?
A: Ampicillin
B: Ceftriaxone
C: Ciprofloxacin
D: Doxycycline
E: Nitrofurantoin
"""

prompts = [
    "What are the early warning signs of stroke and what should I do if I suspect someone is having one?",
    "How do different classes of antidepressants work and what factors determine which medication might be prescribed?",
    "What is the relationship between inflammation, autoimmune conditions, and chronic disease progression?"
]

In [9]:
system_prompt = "You are a helpful medical assistant."

In [10]:
def invoke_realtime_endpoint(record):

    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType="application/json",
        Accept="application/json",
        Body=json.dumps(record),
    )

    return json.load(response["Body"])

#### Chat Completion

In [12]:
input_data = {
    "model": "/opt/ml/model",
    "messages": [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt1},
    ],
    "max_tokens": 2048,
    "temperature": 0.8,
    "top_p": 0.95,
}

result = invoke_realtime_endpoint(input_data)
output_content = result['choices'][0]['message']['content']
print(output_content)

Emerging mRNA technologies and traditional vaccine approaches offer different advantages and mechanisms for disease prevention. Here's a comparison of the two:

### mRNA Vaccines
1. **Mechanism of Action**: 
   - mRNA vaccines use a small piece of the messenger RNA (mRNA) that encodes a disease-specific antigen. Once inside the body, cells use this mRNA to produce the antigen, which then triggers an immune response.
   - This approach is highly adaptable, allowing for rapid development and modification in response to emerging pathogens.

2. **Advantages**:
   - **Speed of Development**: mRNA vaccines can be developed more quickly than traditional vaccines, as seen with COVID-19 vaccines.
   - **Flexibility**: They can be easily modified to target new variants of a virus.
   - **Safety**: mRNA does not integrate into the host genome, reducing the risk of insertional mutagenesis.

3. **Challenges**:
   - **Stability and Storage**: mRNA vaccines often require ultra-cold storage conditions

#### Text Completion

In [14]:
input_data ={
        "model": "/opt/ml/model",
        "prompt": f"{system_prompt}\n\nUser: {prompt2}\n\nAssistant:",
        "max_tokens": 2048,
        "temperature": 0.8,
        "top_p": 0.95,
    }

result = invoke_realtime_endpoint(input_data)
output_text = result['choices'][0]['text']
print(output_text)

 The best treatment for this patient is E: Nitrofurantoin. Nitrofurantoin is the first-line treatment for uncomplicated urinary tract infections in pregnant women. It is safe to use during pregnancy and is effective against the most common bacteria that cause urinary tract infections, including Escherichia coli. Nitrofurantoin is also less likely to cause resistance compared to other antibiotics, which is important to preserve antibiotic effectiveness for future use.


### 3.2 Real-time inference response as a stream via Amazon SageMaker Endpoint

In [16]:
def invoke_streaming_endpoint(record):
    try:
        response = sm_runtime.invoke_endpoint_with_response_stream(
            EndpointName=model_name,
            Body=json.dumps(record),
            ContentType="application/json",
            Accept="text/event-stream"
        )

        for event in response["Body"]:
            if "PayloadPart" in event:
                chunk = event["PayloadPart"]["Bytes"].decode("utf-8")
                if chunk.startswith("data:"):
                    try:
                        data = json.loads(chunk[5:].strip())
                        if "choices" in data and len(data["choices"]) > 0:
                            choice = data["choices"][0]
                            if "text" in choice:
                                yield choice["text"]
                            elif "delta" in choice and "content" in choice["delta"]:
                                yield choice["delta"]["content"]

                    except json.JSONDecodeError:
                        continue 
            elif "ModelStreamError" in event:
                error = event["ModelStreamError"]
                yield f"\nStream error: {error['Message']} (Error code: {error['ErrorCode']})"
                break
            elif "InternalStreamFailure" in event:
                failure = event["InternalStreamFailure"]
                yield f"\nInternal stream failure: {failure['Message']}"
                break
    except Exception as e:
        yield f"\nAn error occurred during streaming: {str(e)}"

#### Chat Completion

In [17]:
payload = {
    "model": "/opt/ml/model",
    "messages": [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt1}
    ],
    "max_tokens": 2048,
    "temperature": 0.8,
    "top_p": 0.95,
    "stream": True
}

for chunk in invoke_streaming_endpoint(payload):
    print(chunk, end="", flush=True)

Emerging mRNA technologies and traditional vaccine approaches both aim to prevent disease, but they differ significantly in their mechanisms, development processes, and certain advantages and disadvantages.

### mRNA Vaccines:
1. **Mechanism**: mRNA vaccines use messenger RNA to instruct cells to produce a protein that is part of the pathogen, prompting an immune response. This approach allows for rapid development and adaptation.
   
2. **Development Speed**: mRNA vaccines can be developed more quickly than traditional vaccines. The COVID-19 pandemic showcased this with the rapid development and deployment of mRNA vaccines.

3. **Adaptability**: They can be easily modified to address new variants or different pathogens, which is particularly useful in rapidly evolving viruses.

4. **Production**: mRNA vaccines can be manufactured quickly and at scale once the genetic sequence of the pathogen is known, without the need for culturing the pathogen itself.

5. **Safety**: Since mRNA does 

#### Text Completion

In [18]:
payload = {
    "model": "/opt/ml/model",
    "prompt": f"{system_prompt}\n\nUser: {prompt2}\n\nAssistant:",
    "max_tokens": 2048,
    "temperature": 0.8,
    "top_p": 0.95,
    "stream": True
}

for chunk in invoke_streaming_endpoint(payload):
    print(chunk, end="", flush=True)

 The best treatment for this patient is E: Nitrofurantoin.

This patient is likely experiencing a urinary tract infection (UTI) during pregnancy. Nitrofurantoin is safe for use during pregnancy and is in treating UTIs. Ampicillin, ceftriaxone, and ciprofloxacin are not recommended during pregnancy due to potential risks to the fetus. Doxycycline is contraindicated during pregnancy due to the risk of fetal harm. It is to consult with a healthcare provider for proper diagnosis and.

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Batch inference

In [20]:
validation_json_file_name1 = "input1.json"

validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/"


def write_and_upload_to_s3(input_data, file_name):
    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/{file_name}",
        Body=(bytes(input_data.encode("UTF-8"))),
    )

In [21]:
input_json_data1 = json.dumps(
    {
        "model": "/opt/ml/model",
        "prompt": [f"{system_prompt}\n\nUser: {prompt}\n\nAssistant:" for prompt in prompts],
        "max_tokens": 2048,
        "temperature": 0.8,
        "top_p": 0.95,
    }
)

write_and_upload_to_s3(input_json_data1, f"{validation_json_file_name1}")

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path,
)
transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [ ]:
from urllib.parse import urlparse

def retrieve_json_output_from_s3(validation_file_name):

    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)
    result = json.loads(response["Body"].read().decode("utf-8"))
    
    for idx, choice in enumerate(result.get("choices", [])):
        print(f"Response {idx + 1}:\n{choice.get('text', '')}\n{'=' * 75}")

In [26]:
retrieve_json_output_from_s3(validation_json_file_name1)

Response 1:
 Early warning signs of stroke include facial drooping, arm weakness, and speech difficulty. If you suspect someone is having a stroke, you should call 911 immediately. The sooner they receive medical attention, the better their chances of recovery. It's important to remember that every minute counts in a stroke situation.
Response 2:
 Antidepressants are commonly used to treat major depressive disorder and other mental health conditions. There are several classes of antidepressants, including Selective Serotonin Reuptake Inhibitors (SSRIs), Serotonin-Norepinephrine Reuptake Inhibitors (SNRIs), Tricyclic Antidepressants (TCAs), and Monoamine Oxidase Inhibitors (MAOIs). 

SSRIs, such as fluoxetine and sertraline, work by blocking the reuptake of serotonin, a neurotransmitter that plays a role in mood regulation. SNRIs, like venlafaxine and duloxetine, block the reuptake of both serotonin and norepinephrine. TCAs, such as amitriptyline and imipramine, block the reuptake of se

Congratulations! You just verified that the batch transform job is working as expected. Since the model is not required, you can delete it. Note that you are deleting the deployable model. Not the model package.

In [ ]:
model.delete_model()

### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

